In [1]:
import spacy
import pandas as pd

import json
import spacy
import requests
import random
import time

import spacy
import tqdm
from collections import Counter

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

import pyLDAvis
import pyLDAvis.gensim

In [2]:
df = pd.read_csv("C:/Users/Rebeca OdC/Documents/A4/Social media analysis/trump_term.csv")
#corpus_text = df['lead_paragraph'].values

In [3]:
df['full_text']=df['headline']+df['lead_paragraph']
corpus_text = df['full_text'].values
df=df.dropna(subset=['full_text'])
df['lenght']=df['full_text'].apply(len)

In [4]:
#CONVERTING INTO LIST OF TOKEMS AND LEMATIZATION
nlp = spacy.load("en_core_web_sm")
corpus_text=df.full_text.values
documents_nlp = []
for d in tqdm.tqdm(nlp.pipe(corpus_text,n_process=5),total=len(corpus_text)):
    documents_nlp.append(d)
docs = [[token.lemma_.lower() for token in doc if  token.is_alpha] for doc in documents_nlp]

#To avoid 'or' and 'of', I remove words that have 2 characters:
docs = [[token for token in doc if len(token) > 2] for doc in docs]

#CREATING DICTIONNARY
id2word = Dictionary(docs)
id2word.filter_extremes(no_below=20, no_above=0.1)
corpus = [id2word.doc2bow(text) for text in docs]
#testing dictionnary
print(corpus[100])

100%|██████████| 1853/1853 [00:20<00:00, 90.12it/s] 


[(118, 1), (145, 1), (164, 1), (255, 2), (302, 1), (324, 1)]


In [5]:
#Finally, the model
lda_model = LdaModel(corpus=corpus,
                   id2word=id2word,
                   num_topics=15,
                   passes=20,
                   chunksize=2000)
from gensim.models.coherencemodel import CoherenceModel
cm = CoherenceModel(model=lda_model, corpus=corpus, coherence='u_mass')
cm.get_coherence()

#docs_2_topics will store the probability distribution of topics over documents.
docs_2_topics = lda_model[corpus]
docs_2_topics[24]

[(4, 0.07656632), (5, 0.37949803), (8, 0.49060023)]

In [6]:
label_dict = {i: ', '.join([token for token, score in lda_model.show_topic(i, topn=10)]) for i in range(0, lda_model.num_topics)}
for topic in label_dict:
    print (topic,':\t', label_dict[topic])

0 :	 kim, nuclear, jong, meeting, venezuela, summit, talk, weapon, program, most
1 :	 global, deal, could, nuclear, set, russia, nation, economic, will, they
2 :	 your, here, sign, briefing, email, get, morning, briefinggood, friday, thursday
3 :	 taiwan, huawei, company, technology, executive, tech, giant, government, move, charge
4 :	 iran, kong, sanction, hong, nuclear, oil, against, iranian, nations, sanctions
5 :	 white, house, tariff, plan, make, security, national, adviser, force, beijing
6 :	 tariff, deal, billion, good, official, two, agreement, economic, other, impose
7 :	 coronavirus, european, she, union, europe, pandemic, back, about, health, currency
8 :	 asia, australia, pacific, military, america, more, their, power, defense, university
9 :	 climate, global, intelligence, change, deal, party, communist, agency, threat, issue
10 :	 when, not, minister, foreign, year, come, prime, time, one, policy
11 :	 south, missile, korean, test, nuclear, military, defense, weapon, af

In [7]:
pyLDAvis.enable_notebook()
visualisation = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, sort_topics=False)
visualisation

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0      0.023600 -0.152532       1        1  5.051459
1      0.061660  0.031280       2        1  7.753987
2     -0.471701 -0.030331       3        1  4.641843
3      0.021604  0.128049       4        1  5.962464
4     -0.005141  0.100431       5        1  7.307601
5      0.051832  0.123724       6        1  8.831351
6      0.010945  0.155560       7        1  8.045108
7      0.002044  0.077575       8        1  5.402260
8      0.067023 -0.032514       9        1  5.383483
9      0.017122  0.070193      10        1  5.634064
10     0.084935 -0.097794      11        1  7.187352
11     0.047340 -0.135379      12        1  6.899851
12     0.036314 -0.145840      13        1  6.681057
13     0.022066 -0.076426      14        1  8.331187
14     0.030356 -0.015998      15        1  6.886935, topic_info=        Term        Freq       Total Category  logprob  loglift
194    south  246.000000  246.000000  Default  30.0000  30.0000
133     your  146.000000  146.000000  Default  29.0000  29.0000
232     here  150.000000  150.000000  Default  28.0000  28.0000
226     sign  166.000000  166.000000  Default  27.0000  27.0000
89      iran  184.000000  184.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
21       one   19.231859  127.436579  Topic15  -4.5532   0.7845
58     month   16.819935   90.554607  Topic15  -4.6872   0.9922
47   tuesday   16.851145  147.363981  Topic15  -4.6854   0.5071
157    about   16.864265  176.947631  Topic15  -4.6846   0.3249
8     russia   16.266013  111.173050  Topic15  -4.7207   0.7535

[705 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
157       1  0.039560  about
157       2  0.056514  about
157       4  0.005651  about
157       6  0.084771  about
157       7  0.022606  about
...     ...       ...    ...
51        6  0.020507    you
51       11  0.020507    you
51       14  0.697226    you
51       15  0.020507    you
133       3  0.989857   your

[2693 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

In [8]:
pyLDAvis.save_html(visualisation, 'trump_topic_modelling_visualisation.html')

In [9]:
topic_2_docs={}
for i, doc_2_topics in enumerate(docs_2_topics):
    for (topic,strength) in doc_2_topics:
        topic_2_docs.setdefault(topic,[]).append((i,strength))

In [10]:
#if we want to find the best article(s) for a topic
topic_number=1
best_doc = sorted(topic_2_docs[topic_number], key=lambda item: item[1], reverse=True)
print(label_dict[topic_number])
for doc in best_doc[:10]:
    print (corpus_text[doc[0]],doc[1])

global, deal, could, nuclear, set, russia, nation, economic, will, they
Will the U.S. Help the Saudis Get a Nuclear Weapon?The last thing the Middle East needs is another country with the potential to build nuclear weapons. Yet that could happen if the United States mishandles Saudi Arabia’s plans to enter the nuclear power business and erect as many as 16 nuclear reactors for electricity generation over 25 years. 0.9611091
U.S. Tax Bill May Inspire Cuts Globally, While Fueling Trade TensionsTo President Trump and congressional Republicans, the overhaul of the tax code that became law on Friday will make the United States a better place to do business. To the rest of the world, it has the potential to challenge the global economic order, creating an uneven playing field and setting off a race among countries to cut corporate taxes. 0.95941865
Russia’s Military Drills Near NATO Border Raise Fears of AggressionWASHINGTON — Russia is preparing to send as many as 100,000 troops to the east